In [1]:
!pip install langchain
!pip install langchain_community
!pip install sentence-transformers
!pip install faiss-cpu
!pip install huggingface_hub
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.1 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
login('Your HuggingFace Api-Key')

## Importing Necessary Library

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts.chat import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers.structured import ResponseSchema
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Loading Data

In [12]:
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

In [13]:
docs = load_pdf('YOur Directory')

## Splitting the Document

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 250,
    separators=["\n\n", "\n", " ", ""]
)

In [15]:
final_doc = text_splitter.split_documents(docs)

In [16]:
final_doc[0].metadata

{'source': '/content/Attention Paper.pdf', 'page': 0}

## Doing Indexing and Storing Indexes

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [18]:
db = FAISS.from_documents(final_doc,embedding_model)

In [19]:
ans = db.similarity_search('What is attention?')

In [ ]:
print(ans)

In [21]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
text = format_docs(docs)

## Importing LLM

In [24]:
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

llama2_model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(llama2_model_name)
model = AutoModelForCausalLM.from_pretrained(
    llama2_model_name,
    device_map="auto",
    torch_dtype="auto"
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 128,
    temperature = 0.7
)
llm = HuggingFacePipeline(pipeline = llm_pipeline)

## Making Retrieval Chain

In [26]:
retriever = db.as_retriever()

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

In [28]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
def get_response(Question):
    response = rag_chain.invoke(Question)
    answer_start = response.find("Answer:")+len("Answer:")
    answer = response[answer_start:].strip()
    return answer

In [ ]:
Question = input("Give Your Query:")
get_response(Question)

In [ ]:
ans = get_response(Question)

In [ ]:
print(ans)